In [1]:
from fileinput import filename
import numpy as np
from copy import deepcopy

import random
import csv
from PIL import Image
from PIL import Image as im
from numpy import asarray
import cv2
import math
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import filedialog
from tkinter.filedialog import askopenfile
from matplotlib import pyplot as plt
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
from skimage.util import random_noise 

In [2]:
root = tk.Tk()
root.title('encryption')

In [3]:
def chooseFile():
    global img
    f_types = [('PNG Files', '*.png')]
    global filename
    filename = filedialog.askopenfilename(filetypes=f_types)
    img = Image.open(filename)
    img = img.resize((300, 200))
    img = ImageTk.PhotoImage(img)
    img_label = tk.Label(image=img)
    img_label.image = img
    img_label.grid(column=1, row=1)

    
#     tk.Label(root,text="SECRET TEXT :",font = ('Helvetica', 18, 'bold')).grid(column= 1 ,row=2,  ipadx = 15 , ipady  = 5)
#     inputtxt = tk.Entry(root)
#     inputtxt.grid(column = 2 , row = 2 , ipadx = 15 , ipady = 5 , padx= 0) 

    enc_btn = tk.Button(root, textvariable=button_text, command=lambda: encrypt(),font=("Raleway", 13), bg="#000000", fg="white", height=2, width=12)
    button_text.set("Encrypt")
    enc_btn.grid(column=1, row=3)
    
    
    

In [4]:
button_text = tk.StringVar()
send_btn_text = tk.StringVar()
send_btn = tk.Button(root, textvariable=send_btn_text, command=lambda: chooseFile(),
                     font=("Raleway", 13), bg="#000000", fg="white", height=2, width=12)
send_btn_text.set("Send")
send_btn.grid(column=1, row=0)

In [5]:
def findsize(s,N):
    while s%N != 0:
        N=N+1
    return s//N


In [6]:
def algrm1(m1,m2,N,divided_images):
    position_values = []
    x_coord = []
    y_coord = []

    for i in range(0,m1):


        for k in range(0,m2):
            pos_i = []
            x = []
            y = []
            for j in range(0,N):
                pos_i.append(divided_images[j][i][k])
                x.append(j)
                y.append(j)

            position_values.append(pos_i)
            x_coord.append(x)
            y_coord.append(y)
    return position_values,x_coord,y_coord


In [7]:
def randMatrix(m1,m2):
    random_matrix = []
    for i in range(0,m1):
        random_matrix1 =  np.random.randint(256,size=(m2))
        random_matrix.append(random_matrix1)
    return random_matrix

In [8]:
def pol_val(cur_i,subi, N ,position_values,x_coord,y_coord,randMat,m2):
    
#     coefficient =np.poly1d( generate_polynomial((t-1)-ni , prime))
#   randVal=(np.polyval(coefficient,subi))%prime
#    tempi=1
#     for i in range(1,ni+1):
#             tempi=(tempi*(subi-i))%prime
#     tempi=(tempi*randVal)%prime
    
    tempix=0
    
    for i in range(0,N):
        tempj=1
        i_val = x_coord[cur_i][i]+1
        
        for j in range(0,N):
            j_val = y_coord[cur_i][j]+1
            if i_val!=j_val:
                subi_i=subi-j_val
                i_j=i_val-j_val
                mul=subi_i/i_j
                tempj=(tempj*mul)%257
#                 print(i," ",j," ",subi," ",abs(subi_i)," ",i_j)
        tempix=(tempix+(position_values[cur_i][i]*tempj))%257
        
    qxi=(tempix)%257
    
#     print(qxi,qxj)
    return int(qxi)^(int(randMat[cur_i//m2][cur_i%m2]))

In [9]:
def openNewWindow(N):
     
    # Toplevel object which will
    # be treated as a new window
    newWindow = tk.Toplevel(root)
 
    # sets the title of the
    # Toplevel widget
    newWindow.title("Shadow Image Window")

    # sets the geometry of toplevel
    newWindow.geometry("1200x300")
 
    # A Label widget to show in toplevel
    tk.Label(newWindow,text ="Shadow Images").pack()
    
    fig = plt.figure(figsize=(15,10))
    row = math.ceil(N / 3)
    column = 3
    
    for i in range(1,N+1):
        fig.add_subplot(row,column,i)
        image = cv2.imread('ShadowImage'+str(i)+'.png')
        plt.imshow(image)
        plt.axis('off')
        plt.title('ShadowImage'+str(i))
    
    canvas = FigureCanvasTkAgg(fig,master = newWindow)
    canvas.draw()
    canvas.get_tk_widget().pack()
    toolbar = NavigationToolbar2Tk(canvas,newWindow)
    toolbar.update()
    canvas.get_tk_widget().pack()
    
    newWindow.state('zoomed')

In [10]:
def binaryToDecimal(binary):
 
    decimal, i = 0, 0
    while(binary != 0):
        dec = binary % 10
        decimal = decimal + dec * pow(2, i)
        binary = binary//10
        i += 1
    return decimal


In [11]:
def encrypt():
    
    
    image = Image.open(filename)
    numpydata = np.asarray(image)
    numpydata=numpydata.astype(int)
    

    boolean=0
    if len(numpydata.shape)==2:
        boolean=0
        maxi=max(max(x) for x in numpydata)
        if maxi==1:
            boolean=-1
            for i in range(0,numpydata.shape[0]):
                for j in range(0,numpydata.shape[1]):
    #                 print(numpydata[i][j])
                    if numpydata[i][j]==1:
                        numpydata[i][j]=255 
            print(numpydata)
    elif len(numpydata.shape)==3:
        boolean=1
    
    N = 10
    m1 = findsize(numpydata.shape[0],N)
   
    N1 = numpydata.shape[0] // m1
    m2 = numpydata.shape[1]
    N=min(m1,N1)
    
    
    m1=max(m1,N1)
    
    
    fig = plt.figure(figsize=(15,10))
    row = math.ceil(N / 3)
    column = 3
    
    divided_images = []
    for j in range(0,N):
        divided_images.append(numpydata[(j)* m1 : ((j)*m1)+m1 , :])
    
    if(boolean > 0):
        b=[]
        g=[]
        r=[]
        for i in range (0,N):
            tempb,tempg,tempr = cv2.split(divided_images[i])
            b.append(tempb)
            g.append(tempg)
            r.append(tempr)
        randMatR= randMatrix(m1,m2)
        randMatG= randMatrix(m1,m2)
        randMatB= randMatrix(m1,m2)
        position_valuesR,x_coordR,y_coordR=algrm1(m1,m2,N,r)
        position_valuesG,x_coordG,y_coordG=algrm1(m1,m2,N,g)
        position_valuesB,x_coordB,y_coordB=algrm1(m1,m2,N,b)
        
#         data=im.fromarray(np.array(randMat).astype(np.uint8))
#         data.save('ShadowImage'+str(0)+'.png')
        for i in range(0,N+1):
            col = 0
            Img = []

            #print(">>>>For ",i,"th n value<<<<")
            for j in range(0,m1):
                I_row =[]
            
                for k in range(0,m2):
                    if(i==0):
                        I_row.append([randMatR[j][k],randMatG[j][k],randMatB[j][k]])
                        
                    else:
                        qxR = pol_val(( (j*m2)+k), i, N ,position_valuesR,x_coordR,y_coordR ,randMatR,m2)
                        print("qxR values:" , qxR)
                        qxG = pol_val(( (j*m2)+k), i, N ,position_valuesG,x_coordG,y_coordG,randMatG,m2)
                        print("qxG values:" , qxG)
                        qxB = pol_val(( (j*m2)+k), i, N ,position_valuesB,x_coordB,y_coordB,randMatB,m2)
                        print("qxB values:" , qxB)
                        
                        I_row.append([qxR,qxG,qxB])
                Img.append(I_row)
        
            Img=np.array(Img)
            #check_writable(Img)
            data=im.fromarray((Img).astype(np.uint8))

            data.save('ShadowImage'+str(i)+'.png')
            print("for i th image ",i, " ", Img)
        text =''
        #text = data_embed(inputtxt,N,m1,m2,boolean)

    else:
        position_values,x_coord,y_coord=algrm1(m1,m2,N,divided_images)
        randMat=randMatrix(m1,m2)

        data=im.fromarray(np.array(randMat).astype(np.uint8))
        data.save('ShadowImage'+str(0)+'.png')


        for i in range(1,N+1):
            col = 0
            Img = []

            #print(">>>>For ",i,"th n value<<<<")
            for j in range(0,m1):
                I_row =[]
                for k in range(0,m2):
                    qxi = pol_val(( (j*m2)+k), i, N ,position_values,x_coord,y_coord ,randMat,m2)
                    I_row.append(qxi)
                Img.append(I_row)

            Img=np.array(Img)
            #check_writable(Img)
            data=im.fromarray((Img).astype(np.uint8))

            data.save('ShadowImage'+str(i)+'.png')
            print("for i th image ",i, " ", Img)
            text =''
        #text = data_embed(inputtxt,N,m1,m2,boolean)
    #print(len(text))
    
    for i in range(1,N+1):
        fig.add_subplot(row,column,i)
        image = cv2.imread('ShadowImage'+str(i)+'.png')
        plt.imshow(image)
        plt.axis('off')
        plt.title('ShadowImage'+str(i))
    

    
    with open("Nandm1andm2.txt","w") as f:
        f.write("'N_value','m1','m2','bool'")
        f.write("\n")
        f.write(str(N)+","+str(m1)+","+str(m2)+","+str(boolean)+",")
        f.write(text)
        
    window_btn = tk.Button(root,text ="View Shadow Images",command = lambda: openNewWindow(N),font=("Raleway", 13), bg="#000000", fg="white", height=2, width=30)
    window_btn.grid(column =1,row=4)

   

In [12]:
canvas = tk.Canvas(root, width=600, height=300)
canvas.grid(columnspan=3, rowspan=3)

root.mainloop()